In [2]:
import pandas as pd
import re

In [3]:

grados = pd.read_csv("grados.csv", sep=";")  # id;nombre;id_area;descripcion;salidas
grado_ofertado = pd.read_csv("grados_ofertados.csv", sep=";")  # id;id_grado;id_facultad
centros_campus = pd.read_csv("centros_campus.csv", sep=",")  # nombre_grado;universidad;tipo;facultad;direccion;telefonos;webs;emails;campus
notas_campus = pd.read_csv("notas_campus.csv", sep=",")  # universidad;grado;grupo;nota;año;convocatoria;campus

def normalizar(texto):
    if pd.isna(texto):
        return ""
    t = str(texto)
    t = re.sub(r"\([^)]*\)", "", t)  # eliminar paréntesis (Moncloa), etc.
    return t

# Aplicar normalización
notas_campus["grado_norm"] = notas_campus["grado"].apply(normalizar)

# Hacemos merge por nombre normalizado (sin paréntesis)
grados_con_campus = grados.merge(
    centros_campus[["nombre", "campus"]],
    on="nombre_norm",
    how="left"
)


notas_grados = notas_campus.merge(
    grados_con_campus[["id", "nombre_norm", "campus_norm"]],
    left_on=["grado_norm", "campus_norm"],
    right_on=["nombre_norm", "campus_norm"],
    how="left"
)

print(f"✅ Coincidencias grado + campus: {notas_grados['id'].notna().sum()} / {len(notas_grados)}")


notas_final = notas_grados.merge(
    grado_ofertado,
    left_on="id",
    right_on="id_grado",
    how="left"
)

nota_corte = notas_final[[
    "id_y", "año", "convocatoria", "nota", "grupo"
]].rename(columns={
    "id_y": "id_grado_ofertado"
})

# Tipos y limpieza
nota_corte["año"] = nota_corte["año"].astype(int)
nota_corte["nota"] = nota_corte["nota"].astype(str).str.replace(",", ".").astype(float)
nota_corte["grupo"] = nota_corte["grupo"].astype(str).str.extract(r"(\d+)").astype(float).astype("Int64")

# Eliminar registros sin grado ofertado
nota_corte = nota_corte.dropna(subset=["id_grado_ofertado"])
nota_corte["id_grado_ofertado"] = nota_corte["id_grado_ofertado"].astype(int)

# Añadir id autoincremental
nota_corte.insert(0, "id", range(1, len(nota_corte) + 1))


nota_corte = nota_corte[["id", "id_grado_ofertado", "año", "convocatoria", "nota", "grupo"]]
nota_corte.to_csv("nota_corte.csv", sep=";", index=False, encoding="utf-8")




KeyError: 'grado'